In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-241039
Azure region: eastus2
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-241039


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EW33BLPXD to authenticate.


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster-udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster in the provided workspace, so we use it.')
except ComputeTargetException:
    comp_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=3)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, comp_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
    "--C" : uniform(0.0,10.0),
    "--max_iter" : choice(10,20,30,40,50,60,70,80,90,100,150)
    }
)

# Specify a Policy ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, 
                      evaluation_interval = 2,
                      delay_evaluation = 10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=cluster_name,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy. ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='f1_score',
                             primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs = 10,
                             max_concurrent_runs = 2)



In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config) 

#show run details with the widget.
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [44]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")

print(best_run)

best_run_params = best_run.get_details()['runDefinition']['parameters']
best_run_metrics = best_run.get_metrics()

print("Information about the best model: \n")
print("Run id: {}".format(best_run.id))
print("F1-Score: {}".format(best_run_metrics['f1_score']))
print("Hyperparameter values: \n")
print(dict({best_run_params[1::2][0]['name']: best_run_params[1::2][0]['value']}))
print(dict({best_run_params[1::2][1]['name']: best_run_params[1::2][1]['value']}))

model_name = "HyperDrive_model.pkl"
file_name = "outputs/" + model_name

#if not os.path.exists('outputs/'):
#    os.makedirs('outputs/')
#joblib.dump(value=best_run, filename=file_name)
description = 'HyperDrive best forescasting model'
model = best_run.register_model(model_name='HyperDriveModel', description=description, model_path=file_name)
print(model)

The best hyperparameter values are: 

{'inputs.--C': '0.4514268843771162'}
{'inputs.--max_iter': '30'}


TypeError: cannot pickle '_queue.SimpleQueue' object

In [42]:
print(dict({parameter_values[1::2][0]['name']: parameter_values[1::2][0]['value']}))
print(dict({parameter_values[1::2][1]['name']: parameter_values[1::2][1]['value']}))

{'inputs.--C': '0.4514268843771162'}
{'inputs.--max_iter': '30'}


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
initial_data = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(initial_data)



In [ ]:
y.value_counts()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset, Datastore

# Split data into train and test sets. Set random state to ensure that we have the same partitions for HyperDrive and AutoML
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

# Register the train and test sets
datastore_ws = Datastore.get(ws, 'workspaceblobstore')
train_ds = Dataset.Tabular.register_pandas_dataframe(train_df, datastore_ws, "train data", show_progress=True)
test_ds = Dataset.Tabular.register_pandas_dataframe(test_df, datastore_ws, "test data", show_progress=True)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric = "f1_score",
    training_data=train_ds,
    validation_data=test_ds,
    label_column_name='y',
    n_cross_validations=4,
    compute_target=cpu_cluster)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run = automl_run.get_best_child()

model_name = automl_best_run.properties['model_name']
description = 'AutoML  best forecasting model'
tags = None

model = run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

joblib.dump(automl_best_run, 'outputs/AutoML_model.pkl')

In [ ]:
cpu_cluster.delete()

In [ ]:
# Check status of the cluster for deletion 
cpu_cluster